In [1]:
# http://proceedings.mlr.press/v101/huang19a/huang19a.pdf
# https://www.researchgate.net/publication/220875351_Generative_Models_for_Labeling_Multi-object_Configurations_in_Images
# https://www.tensorflow.org/datasets/catalog/open_images_v4
# Auto-Encoding Progressive Generative Adversarial Networks For 3D Multi Object Scenes

In [2]:
%config Completer.use_jedi = False
from ipywidgets import IntProgress
import matplotlib.pyplot as plt
from tensorflow.keras import layers, losses
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import logging
import tensorflow_datasets as tfds
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.mixture import GaussianMixture
import os

In [3]:
batch_size = 32
epochs = 10
dataset_name = 'scene_parse150'

In [4]:
train_ds,test_ds = tfds.load(name=dataset_name,split=['train', 'test']\
                                           ,as_supervised=False,download=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

ConnectionError: HTTPConnectionPool(host='placeschallenge.csail.mit.edu', port=80): Max retries exceeded with url: /data/ChallengeData2017/annotations_instance.tar (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7faff96cfef0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
dims = [x['image'].get_shape().as_list() for x in train_ds]
dims_df= pd.DataFrame.from_records(data=dims,columns=['height','width','depth'])
dims_df.describe()

In [ ]:
height = 2**(int(np.log2(min(dims_df['height']))))
width = 2**(int(np.log2(min(dims_df['width']))))
height,width = min(height,width),min(height,width)
height,width

In [ ]:
train_ds = train_ds.map(lambda x: tf.image.resize(images=tf.cast(x['image'],dtype=tf.float32)/255.,\
                                                  size=[height,width]))
train_ds = train_ds.batch(batch_size,drop_remainder=True)
train_ds_double_zipped = tf.data.Dataset.zip(datasets=(train_ds,train_ds))

###
test_ds = test_ds.map(lambda x: tf.image.resize(tf.cast(x['image'],dtype=tf.float32)/255.,\
                                                size=[height,width]))
test_ds = test_ds.batch(batch_size,drop_remainder=True)
test_ds_double_zipped = tf.data.Dataset.zip(datasets=(test_ds,test_ds))

###
# validation_ds = validation_ds.map(lambda x: tf.image.resize(tf.cast(x['image'],dtype=tf.float32)/255.\
#                                                             ,size=[height,width]))
# validation_ds = validation_ds.batch(batch_size,drop_remainder=True)
# validation_ds_double_zipped = tf.data.Dataset.zip(datasets=(validation_ds,validation_ds))


In [ ]:
# just to detect the dimension distribution
# dims = [x.get_shape().as_list() for x in train_ds]
# dims_df= pd.DataFrame.from_records(data=dims,columns=['batch','height','width','depth'])
# dims_df.describe()

In [ ]:
for e in train_ds.take(10):
    plt.imshow(e[0].numpy())
    plt.show()

In [ ]:
for e in test_ds.take(10):
    plt.imshow(e[0].numpy())
    plt.show()

In [ ]:
latent_dim = 4096
batch_size = 32
class CAE(tf.keras.Model):
    """Convolutional variational autoencoder."""

    def __init__(self, latent_dim):
        super(CAE, self).__init__()
        self.latent_dim = latent_dim
        self.logger = logging.getLogger('CAE')
        self.encoder = tf.keras.Sequential(name='encoder',layers=\
            [
                tf.keras.layers.InputLayer(input_shape=(height, width, 3)),
                tf.keras.layers.Conv2D(
                    filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
                tf.keras.layers.Conv2D(
                    filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
                tf.keras.layers.Flatten(),
                # No activation
                tf.keras.layers.Dense(latent_dim),
            ]
        )

        self.decoder = tf.keras.Sequential(name='decoder',layers=\
            [
                tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
                tf.keras.layers.Dense(units=int(height/4) * int(width/4) * 32, activation=tf.nn.relu),
                tf.keras.layers.Reshape(target_shape=(int(height/4), int(width/4), 32)),
                tf.keras.layers.Conv2DTranspose(
                    filters=64, kernel_size=3, strides=2, padding='same',
                    activation='relu'),
                tf.keras.layers.Conv2DTranspose(
                    filters=32, kernel_size=3, strides=2, padding='same',
                    activation='relu'),
                # No activation
                tf.keras.layers.Conv2DTranspose(
                    filters=3, kernel_size=3, strides=1, padding='same'),
            ]
        )

        self.encoder.summary()
        self.decoder.summary()
        
        
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


cae = CAE(latent_dim)
cae.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
model_file_path = f'./models/cae_dataset_{dataset_name}_z_dim_{latent_dim}'
print(f'model path = {model_file_path}')

In [ ]:

if os.path.exists(model_file_path):
    print('loading saved model')
    cae = tf.keras.models.load_model(filepath=model_file_path)
else:
    print('building model')
    cae.fit(x=train_ds_double_zipped,validation_data=test_ds_double_zipped,epochs=10)
    print('saving model')
    cae.save(filepath=model_file_path)

In [ ]:
z_tensor = None

inf_or_unknown_cardinality = ((test_ds.cardinality()==tf.data.INFINITE_CARDINALITY)\
    or (test_ds.cardinality() == tf.data.UNKNOWN_CARDINALITY)).numpy()
batches = test_ds.cardinality().numpy() if not inf_or_unknown_cardinality else 500

with tqdm(total=batches) as pbar:
    for batch in test_ds.take(batches):
        z = cae.encoder(batch).numpy()
        if z_tensor is None:
            z_tensor = tf.convert_to_tensor(z)
        else:
            z_tensor = tf.concat([z_tensor,tf.convert_to_tensor(z)],axis=0)
        pbar.update(1)
        #print(f'z shape {z.shape}')
        decoded_imgs = cae.decoder(z).numpy()
        #print(f'decoded images shape {decoded_imgs[0].shape}')
        #plt.imshow(batch[0])
        #plt.show()
        #plt.imshow(decoded_imgs[0])
        #plt.show()
z_tensor.shape

In [ ]:
z_np= z_tensor.numpy()
n_z = z_np.shape[0]
n_z_train = int(0.8*n_z)
z_train = z_np[:n_z_train]
z_test = z_np[n_z_train:]

In [ ]:
print(f"""For Dateset "{dataset_name}" Calculating relative difference of log likelihood  """)
print('############################  ')
g_fit = GaussianMixture(n_components=1,covariance_type='diag').fit(z_train)
logp_g = g_fit.score(X=z_test)
for k in [10,20,50,70,100,200]:
    try:
        gm_fit = GaussianMixture(n_components=k,covariance_type='diag').fit(z_train)
        logp_gm = gm_fit.score(X=z_test)

        rel_diff_logps = (logp_gm- logp_g) / np.abs(logp_g)
        print(f'logp Gaussin Mixture with k = {k} = {logp_gm}  ')
        print(f'logp Gaussian Diagonal = {logp_g}  ')
        print(f'At k = {k} , rel_diff for logps = {rel_diff_logps}  ')
        print('##############  ')
    except Exception as e:
        print(f'Catched expection {e}  ')

## Results

**For dataset VOC**

For Dateset VOC Calculating relative difference of log likelihood  
############################  
logp Gaussin Mixture with k = 10 = 3239.302399019124  
logp Gaussian Diagonal = 2790.16976645871  
At k = 10 , rel_diff for logps = 0.16096964348174925  
##############  
logp Gaussin Mixture with k = 20 = 3334.117578802105  
logp Gaussian Diagonal = 2790.16976645871  
At k = 20 , rel_diff for logps = 0.19495151115259016  
##############  
logp Gaussin Mixture with k = 50 = 3411.8171131494596  
logp Gaussian Diagonal = 2790.16976645871  
At k = 50 , rel_diff for logps = 0.22279911214138984  
##############  
logp Gaussin Mixture with k = 70 = 3429.9682203595194  
logp Gaussian Diagonal = 2790.16976645871  
At k = 70 , rel_diff for logps = 0.22930448949450236  
##############  
logp Gaussin Mixture with k = 100 = 3428.6442835155385  
logp Gaussian Diagonal = 2790.16976645871  
At k = 100 , rel_diff for logps = 0.22882998903223795  
##############  
logp Gaussin Mixture with k = 200 = 3415.867706729245  
logp Gaussian Diagonal = 2790.16976645871  
At k = 200 , rel_diff for logps = 0.22425084946163418  
##############  

**For dataset wider_face**
For Dateset wider_face Calculating relative difference of log likelihood  
############################  
logp Gaussin Mixture with k = 10 = 1726.1243177708377  
logp Gaussian Diagonal = 1561.8860242887904  
At k = 10 , rel_diff for logps = 0.10515382744194393  
##############  
logp Gaussin Mixture with k = 20 = 1751.7959812169918  
logp Gaussian Diagonal = 1561.8860242887904  
At k = 20 , rel_diff for logps = 0.1215901506095347  
##############  
logp Gaussin Mixture with k = 50 = 1776.35932923814  
logp Gaussian Diagonal = 1561.8860242887904  
At k = 50 , rel_diff for logps = 0.13731687307145898  
##############  
Catched expection Fitting the mixture model failed because some components have ill-defined empirical covariance (for instance caused by singleton or collapsed samples). Try to decrease the number of components, or increase reg_covar.  
Catched expection Fitting the mixture model failed because some components have ill-defined empirical covariance (for instance caused by singleton or collapsed samples). Try to decrease the number of components, or increase reg_covar.  
Catched expection Fitting the mixture model failed because some components have ill-defined empirical covariance (for instance caused by singleton or collapsed samples). Try to decrease the number of components, or increase reg_covar.  



For Dateset "mnist" Calculating relative difference of log likelihood  
############################  
logp Gaussin Mixture with k = 10 = 7841.956234385311  
logp Gaussian Diagonal = 7377.442821602376  
At k = 10 , rel_diff for logps = 0.06296401395653828  
##############  
logp Gaussin Mixture with k = 20 = 7988.197403977028  
logp Gaussian Diagonal = 7377.442821602376  
At k = 20 , rel_diff for logps = 0.08278675919876485  
##############  
logp Gaussin Mixture with k = 50 = 8243.583374395243  
logp Gaussian Diagonal = 7377.442821602376  
At k = 50 , rel_diff for logps = 0.11740389912025657  
##############  
logp Gaussin Mixture with k = 70 = 8305.125097353703  
logp Gaussian Diagonal = 7377.442821602376  
At k = 70 , rel_diff for logps = 0.12574577644098017  
##############  
logp Gaussin Mixture with k = 100 = 8385.913961674707  
logp Gaussian Diagonal = 7377.442821602376  
At k = 100 , rel_diff for logps = 0.1366965714894272  
##############  
logp Gaussin Mixture with k = 200 = 8500.999476912375  
logp Gaussian Diagonal = 7377.442821602376  
At k = 200 , rel_diff for logps = 0.15229622004253812  
##############  

For Dateset "cifar100" Calculating relative difference of log likelihood  
############################  
logp Gaussin Mixture with k = 10 = 4348.738804362613  
logp Gaussian Diagonal = 4039.8092947584987  
At k = 10 , rel_diff for logps = 0.07647131016925454  
##############  
logp Gaussin Mixture with k = 20 = 4384.332721210587  
logp Gaussian Diagonal = 4039.8092947584987  
At k = 20 , rel_diff for logps = 0.08528210153362795  
##############  
logp Gaussin Mixture with k = 50 = 4422.78147918955  
logp Gaussian Diagonal = 4039.8092947584987  
At k = 50 , rel_diff for logps = 0.09479957010048548  
##############  
logp Gaussin Mixture with k = 70 = 4428.119142994677  
logp Gaussian Diagonal = 4039.8092947584987  
At k = 70 , rel_diff for logps = 0.09612083638205283  
##############  
logp Gaussin Mixture with k = 100 = 4425.925444793071  
logp Gaussian Diagonal = 4039.8092947584987  
At k = 100 , rel_diff for logps = 0.09557781614482229  
##############  
logp Gaussin Mixture with k = 200 = 4424.68448280697  
logp Gaussian Diagonal = 4039.8092947584987  
At k = 200 , rel_diff for logps = 0.09527063283601832  
##############  